In [12]:
from google.cloud import bigquery
bq = bigquery.Client()

In [13]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [14]:
#q1

q1 = bq.query(
"""
SELECT geo_id 
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""")
df = q1.to_dataframe()
result = df['geo_id'].values[0]
result

'55025'

In [15]:
#q2

q2 = bq.query(
"""
SELECT state_fips_code, COUNT(*) AS num_counties
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY num_counties DESC
LIMIT 5
""")
df = q2.to_dataframe()
dict = df.set_index('state_fips_code')['num_counties'].to_dict()
dict

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [24]:
#q3

from google.cloud import bigquery
q1 = bq.query(
"""
SELECT geo_id 
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""",
job_config=bigquery.QueryJobConfig(use_query_cache=False))

q2 = bq.query(
"""
SELECT state_fips_code, COUNT(*) AS num_counties
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY num_counties DESC
LIMIT 5
""",
job_config=bigquery.QueryJobConfig(use_query_cache=False))

cost1 = q1.total_bytes_billed / 1024**4
cost1 *= 6.25

cost2 = q2.total_bytes_billed / 1024**4
cost2 *= 6.25

cost_dict = {'q1': cost1, 'q2': cost2}
cost_dict

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}